04/20/2020

need to annotate all anchors and all 2kb promoter regions


make files in processed/annon


In [2]:
import os,glob
import pybedtools
import pandas as pd
from collections import Counter,defaultdict

In [3]:
test = pybedtools.BedTool('../data/external/promoter_hg19_2000_0_sort.bed')


In [31]:
anchor = pybedtools.BedTool('../data/interim/merged/anchors_bed_sort/GDSD0_sort.bed')


In [6]:
footprint = pybedtools.BedTool('../data/interim/merged/footprinting/GDSD0_merged.bed')

In [17]:
atac = pybedtools.BedTool('../data/interim/merged/atac/GDSD0_merged.bed')

In [33]:
test_anchor = test.intersect(anchor, wa=True, wb=True, sorted=True, names='anchor')
test_anchor.head()

chr1	899877	901877	PLEKHN1	chr1	900000	905000	chr1_900000_905000
 chr1	915497	917497	C1orf170	chr1	915000	920000	chr1_915000_920000
 chr1	933552	935552	HES4	chr1	935000	940000	chr1_935000_940000
 chr1	946803	948803	ISG15	chr1	945000	950000	chr1_945000_950000
 chr1	953503	955503	AGRN	chr1	955000	960000	chr1_955000_960000
 chr1	1049741	1051741	C1orf159	chr1	1045000	1050000	chr1_1045000_1050000
 chr1	1207265	1209265	UBE2J2	chr1	1205000	1210000	chr1_1205000_1210000
 chr1	1241947	1243947	PUSL1	chr1	1240000	1245000	chr1_1240000_1245000
 chr1	1242989	1244989	ACAP3	chr1	1240000	1245000	chr1_1240000_1245000
 chr1	1308875	1310875	AURKAIP1	chr1	1310000	1315000	chr1_1310000_1315000
 

In [35]:
test_atac = test.intersect(atac, wa=True, wb=True, sorted=True, names='atac')
print(test_atac.head())


chr1	858260	860260	SAMD11	chr1	859065	859355
 chr1	858260	860260	SAMD11	chr1	859065	859355
 chr1	892689	894689	NOC2L	chr1	894393	894868
 chr1	892689	894689	NOC2L	chr1	894393	894868
 chr1	892689	894689	NOC2L	chr1	894393	894868
 chr1	893967	895967	KLHL17	chr1	894393	894868
 chr1	893967	895967	KLHL17	chr1	894393	894868
 chr1	893967	895967	KLHL17	chr1	894393	894868
 chr1	893967	895967	KLHL17	chr1	895834	896065
 chr1	893967	895967	KLHL17	chr1	895834	896065
 None


In [36]:
test_atac.head()

chr1	858260	860260	SAMD11	chr1	859065	859355
 chr1	858260	860260	SAMD11	chr1	859065	859355
 chr1	892689	894689	NOC2L	chr1	894393	894868
 chr1	892689	894689	NOC2L	chr1	894393	894868
 chr1	892689	894689	NOC2L	chr1	894393	894868
 chr1	893967	895967	KLHL17	chr1	894393	894868
 chr1	893967	895967	KLHL17	chr1	894393	894868
 chr1	893967	895967	KLHL17	chr1	894393	894868
 chr1	893967	895967	KLHL17	chr1	895834	896065
 chr1	893967	895967	KLHL17	chr1	895834	896065
 

In [203]:
# test_foot = test.intersect(footprint,wa=True, wb=True, sorted=True, names='footprinting',f=2.5E-6)
test_foot = test.intersect(footprint,wo=True, sorted=True, names='footprinting',f=2.5E-6)

# test_foot = test_foot.merge(o)

In [204]:
test_foot.head()

chr1	936803	938803	ISG15	chr1	937407	937424	ZN134_HUMAN.H11MO.1.C	10.4906703374	-	17
 chr1	936803	938803	ISG15	chr1	937407	937419	VEZF1_HUMAN.H11MO.1.C	10.990834234	-	12
 chr1	936803	938803	ISG15	chr1	937408	937423	ZIC2_MOUSE.H11MO.0.C	15.1361938044	+	15
 chr1	936803	938803	ISG15	chr1	937408	937423	ZIC3_MOUSE.H11MO.0.A	16.2663948899	+	15
 chr1	936803	938803	ISG15	chr1	937408	937423	ZIC2_HUMAN.H11MO.0.D	14.5972131949	+	15
 chr1	936803	938803	ISG15	chr1	937408	937418	FEV_HUMAN.H11MO.0.B	10.7016784351	-	10
 chr1	936803	938803	ISG15	chr1	937408	937423	ZIC3_HUMAN.H11MO.0.B	16.2663948899	+	15
 chr1	936803	938803	ISG15	chr1	937408	937418	FEV_MOUSE.H11MO.0.B	10.7016784351	-	10
 chr1	936803	938803	ISG15	chr1	937410	937434	ZN436_HUMAN.H11MO.0.C	9.20997622916	-	24
 chr1	936803	938803	ISG15	chr1	937411	937431	ZN263_HUMAN.H11MO.0.A	10.4882011972	-	20
 

# testing process_crms

## A. for the 2KB region upstream of the TSS

In [2]:
THRES=1

In [3]:
def get_exp(rna_df, sym, tissue,verbose=True):
    try:
        return rna_df.loc[sym,tissue]
    except KeyError:
        if verbose:
            print(sym, 'not found')
        return 0

In [13]:
rna_tpm_file = '../data/interim/rna/tissue_tpm_sym.csv'
rna_df = pd.read_csv(rna_tpm_file, index_col=0,header=0)
rna_df.index = [x.upper() for x in rna_df.index.values]
rna_df[:5]

,A431-CTRLi,A431-p63i,Airway,Astrocytes,Bladder,CAL27-CTRLi,CAL27-p63i,COLO_SCR_DMSO,COLO_SCR_PLX,COLO_shMITF_DMSO,...,SKMEL5_SCR_DMSO,SKMEL5_SCR_PLX,SKMEL5_shMITF-DMSO,SKMEL5_shMITF_PLX,Thyroid,Uterine,WM_SCR_DMSO,WM_SCR_PLX,WM_shMITF_DMSO,WM_shMITF_PLX
A1BG,3.740,3.965,3.575,18.66,3.31,0.100,0.055,48.420,49.885,42.995,...,55.475,70.930,47.805,83.920,0.390,2.425,92.060,142.880,112.345,145.610
A1BG-AS1,0.405,0.045,0.055,0.43,0.03,0.165,0.990,0.130,0.120,0.135,...,0.365,0.405,0.280,0.140,0.055,0.030,0.380,0.405,0.325,0.390
A1CF,0.000,0.000,0.005,0.01,0.01,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000
A2M,0.025,0.060,0.000,50.67,1.10,0.150,0.235,37.565,72.670,37.580,...,64.130,89.220,106.345,193.725,0.105,0.950,54.645,79.480,62.120,68.885
A2M-AS1,0.000,0.005,0.025,0.07,0.00,0.010,0.000,0.010,0.080,0.025,...,0.055,0.030,0.040,0.095,0.000,0.000,0.050,0.030,0.050,0.135


In [14]:
rna_df.loc['A1BG','COLO_SCR_PLX']

49.885

In [15]:
# rna_df.columns.sort_values()

In [16]:
annon_file = '../data/external/HOCOMOCOv11_annotation.csv'
annon_df = pd.read_csv(annon_file)
annon_df[['motif_abbr','motif_info']]=annon_df['id'].str.split("_",expand=True)
annon_dict = pd.Series(annon_df.tf.values, index=annon_df.motif_abbr.values).to_dict()


In [17]:
# filepath  = '../data/interim/annon/promoter_footprinting/promoter_GDSD3_annon.bed'
# filepath  = '../data/interim/annon/promoter_footprinting/promoter_COLO_SCR_PLX_annon.bed'
filepath  = '../data/interim/annon/promoter_footprinting/promoter_CAL27-CTRLi_annon.bed'
filename = os.path.basename(filepath)
foot_df = pd.read_csv(filepath,sep='\t',
            header=None)
tissue = filename.split('_annon.bed')[0].split("_")[-1]
if tissue not in list(rna_df.columns.values):
    raise ValueError('tissue not found in rna matrix', tissue)
foot_df.columns = ['chr_p','start_p','stop_p','TSS','chr_f','start_f','stop_f','motif_id','score','-']
foot_df['p_loc'] = foot_df.chr_p + '_' + foot_df.start_p.map(str) + '_' + foot_df.stop_p.map(str) 

foot_df['f_loc'] = foot_df.chr_f + '_' + foot_df.start_f.map(str) + '_' + foot_df.stop_f.map(str) 
foot_df[['motif_abbr','motif_info']]=foot_df['motif_id'].str.split("_",expand=True)
foot_df['animal']=foot_df['motif_info'].str.split(".",expand=True).loc[:,0]
print(foot_df.shape)
foot_df = foot_df[foot_df.animal=='HUMAN']
print(foot_df.shape)
foot_df['TF'] = foot_df.motif_abbr.map(annon_dict)
print(foot_df.shape)
foot_df = foot_df[~foot_df.TF.isna()]
print(foot_df.shape)





(106390, 15)
(60834, 15)
(60834, 16)
(47008, 16)


In [9]:
pd.Series(foot_df.TF.unique()).isin(rna_df.index).value_counts()

True     398
False      3
dtype: int64

In [10]:
foot_df['TF_exp']= foot_df.TF.apply(lambda x: get_exp(rna_df, x, tissue,verbose=False))

In [11]:
foot_df_filt = foot_df[foot_df.TF_exp>THRES]
foot_df_filt = foot_df_filt[['TSS','TF','p_loc']]
foot_df.shape,foot_df_filt.shape

((47008, 17), (0, 3))

In [240]:
foot_df_filt_count = pd.DataFrame({'count':foot_df_filt.groupby('TSS')['TF'].value_counts()}).reset_index()

In [241]:
foot_df_filt_count_pivot = foot_df_filt_count.pivot(index='TSS',columns='TF',values='count').fillna(0)

In [242]:
foot_df_filt_count_pivot.shape

(1846, 230)

In [243]:
foot_df_filt_count_pivot.index

Index(['ABCA7', 'ABCB6', 'ABCC5', 'ABCF3', 'ABHD12B', 'AC003101.1',
       'AC004017.1', 'AC004466.1', 'AC004817.1', 'AC005606.1',
       ...
       'ZNF581', 'ZNF622', 'ZNF629', 'ZNF672', 'ZNF704', 'ZNF771', 'ZNF773',
       'ZNF821', 'ZP4', 'ZSCAN9'],
      dtype='object', name='TSS', length=1846)

In [254]:
'MYC' in list(foot_df_filt_count_pivot.index)

False

In [558]:
foot_df_pro = get_foot_df('../data/interim/annon/promoter_footprinting/promoter_GDSD3_annon.bed', annon_dict, rna_df, thres=1,verbose=False)

foot_count_pro = get_foot_counts(foot_df_pro)


## B. for the 5KB anchor region closest to to the TSS 
Case 1: TSS in 5KB region --> just use that region
Case 2: TSS not in 5KB region --> find closest region, that is maximum 5kb away

this is just a function version of the above cmds

In [277]:
def get_foot_df(foot_file, annon_dict, rna_df, thres=1,verbose=False):
    """
    from footprinting annotation file, annotate with motif/TF (rna and transcription factor info) 

    Arguments:
        foot_file  = <str> to filepath i.e. '../data/interim/annon/promoter_footprinting/promoter_GDSD0_annon.bed'
        annon_dict = dict from `get_tf_dict`
        rna_df = DataFrame from `read_rna_file`, filter by expression motif --> TF
        thres = <int> for considering what is expressed

    Return: 
        foot_df_filt = <DataFrame> with columns, ['TSS','TF', 'p_loc']
            indicating the TSS (gene) the TF is annotated for and the region annotated in`
                p_loc = <str> with <chr>_<start>_<stop>, can correspond to anchor region
    """
    # read in footprinting file
    filename = os.path.basename(foot_file)
    foot_df = pd.read_csv(foot_file,sep='\t',
                header=None)
    tissue = filename.split('_annon.bed')[0].split("_")[-1]
    foot_df.columns = ['chr_p','start_p','stop_p','TSS','chr_f','start_f','stop_f','motif_id','score','-']
    foot_df['p_loc'] = foot_df.chr_p + '_' + foot_df.start_p.map(str) + '_' + foot_df.stop_p.map(str)
    foot_df['f_loc'] = foot_df.chr_f + '_' + foot_df.start_f.map(str) + '_' + foot_df.stop_f.map(str)
    foot_df[['motif_abbr','motif_info']]=foot_df['motif_id'].str.split("_",expand=True)
    foot_df['animal']=foot_df['motif_info'].str.split(".",expand=True).loc[:,0]
    # human tfs only
    foot_df = foot_df[foot_df.animal=='HUMAN']

    # mapping motif --> TF with the annotation dictionary
    foot_df['TF'] = foot_df.motif_abbr.map(annon_dict)
    foot_df = foot_df[~foot_df.TF.isna()]

    # get expression from rna_df
    foot_df['TF_exp']= foot_df.TF.apply(lambda x: get_exp(rna_df, x, tissue,verbose=verbose))
    foot_df_filt = foot_df[foot_df.TF_exp>thres]
    foot_df_filt = foot_df_filt[['TSS','TF', 'p_loc']]


    return foot_df_filt


def get_foot_counts(foot_df):
    """
    from a foot_df from `get_foot_df`
    get `foot_df_count_pivot` that is a mxn DataFrame
        with m = # of genes annotated in foot file, and
            n = # of tfs annotated from `annon_dict`
            filtered through expression,

    Arguments:
        foot_df: <DataFrame> wit at least required columns TSS, TF (all other columns are ignored)
        

    """
    # pivoting table to create counts DataFrame
    foot_df_filt_count = pd.DataFrame({'count':foot_df.groupby('TSS')['TF'].value_counts()}).reset_index()
    foot_df_filt_count_pivot = foot_df_filt_count.pivot(index='TSS',columns='TF',values='count').fillna(0)
    return foot_df_filt_count_pivot

In [278]:
foot_file = '../data/interim/annon/anchor_footprinting/GDSD3_annon.bed'
foot_df = get_foot_df(foot_file, annon_dict, rna_df, thres=1,verbose=False)
foot_df = foot_df[['TF','p_loc']]


In [292]:
foot_df[:5]

,TF,p_loc
5,THAP1,chr1_710000_715000
7,ZBTB14,chr1_710000_715000
8,JUND,chr1_845000_850000
10,JUN,chr1_845000_850000
12,FOSL1,chr1_845000_850000


In [293]:
foot_df[foot_df.p_loc=='chr8_128735000_128740000']

,TF,p_loc


In [3]:
%%time
anchor_file = '../data/interim/annon/promoter_anchors/promoter_GDSD3_annon.bed'
anchor_annon_df = pd.read_csv(anchor_file,sep='\t',
            header=None)
anchor_annon_df.columns = ['chr_p','start_p','stop_p','TSS','chr_f','start_f','stop_f','anchor','overlap']
anchor_annon_df_filt = anchor_annon_df.groupby('TSS').apply(lambda x: x.nlargest(1, "overlap")).reset_index(drop=True)
anchor_annon_df_filt = anchor_annon_df_filt[['anchor','TSS']]

CPU times: user 19.4 s, sys: 241 ms, total: 19.7 s
Wall time: 19.6 s


In [422]:
foot_df_anchor = foot_df.merge(anchor_annon_df_filt,how='inner', left_on='p_loc',right_on='anchor')

In [362]:
anchor_annon_df.TSS.nunique()

10397

In [363]:
anchor_annon_df_filt.TSS.nunique()

10397

In [364]:
'MYC' in list(anchor_annon_df_filt.TSS)

True

In [365]:
anchor_annon_df[anchor_annon_df.TSS=='MYC']

,chr_p,start_p,stop_p,TSS,chr_f,start_f,stop_f,anchor,overlap
18025,chr8,128732680,128742680,MYC,chr8,128730000,128735000,chr8_128730000_128735000,2320
18026,chr8,128732680,128742680,MYC,chr8,128735000,128740000,chr8_128735000_128740000,5000
18027,chr8,128732680,128742680,MYC,chr8,128740000,128745000,chr8_128740000_128745000,2680


In [366]:
anchor_annon_df_filt[:5]

,anchor,TSS
0,chr12_8960000_8965000,A2ML1
1,chr1_33770000_33775000,A3GALT2
2,chr22_43110000_43115000,A4GALT
3,chr3_137840000_137845000,A4GNT
4,chr12_53710000_53715000,AAAS


In [309]:
foot_df_anchor.TSS.nunique()

3325

In [559]:
foot_count_anchor = get_foot_counts(foot_df_anchor)
foot_count_anchor.shape

(3325, 230)

This is an increase from the (1846, 230) anchors previously noted

In [310]:
'TP63' in list(foot_df_anchor.TSS)

False

## C. for the 5KB regions next to closest anchors of TSS --> find all looped regions


In [4]:
%%time
# in func make_loop_df
loop_file = '../data/interim/merged/loops/GDSD3.loops.csv'
loop_df = pd.read_csv(loop_file,index_col=0)
loop_df_rev = loop_df.copy()
loop_df_rev.columns = ['target','source','count']
loop_df_bi = pd.concat([loop_df, loop_df_rev])
# target is the "loop"
loop_df_bi = loop_df_bi.merge(anchor_annon_df_filt,how='inner',left_on='source',right_on='anchor' )
foot_df_loop = foot_df.merge(loop_df_bi,how='inner', left_on='p_loc',right_on='target')
foot_count_loop = get_foot_counts(foot_df_loop)
foot_count_loop.shape



/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


NameError: name 'foot_df' is not defined

In [372]:
'TP63' in list(foot_df_loop.TSS)

True

# D. atac_regions

In [1]:
def read_atac_file(atac_file):
    """
    read in to DataFrame
    atac_file = '../data/interim/annon/promoter_atac/promoter_GDSD3_annon.bed'
    and get # of atac regions per tss
    
    output: atac_count <DataFrame> with `TSS` and index and  `num_atac_regions` as columns 
    """
    filename = os.path.basename(atac_file)
    tissue = filename.split('_annon.bed')[0].split("_")[-1]
    atac_df = pd.read_csv(atac_file,sep='\t', header=None)
    atac_df.columns = ['chr_p','start_p','stop_p','TSS','chr_a','start_a','stop_a']
    atac_df['p_loc'] = atac_df.chr_p + '_' + atac_df.start_p.map(str) + '_' + atac_df.stop_p.map(str)
    atac_df['a_loc'] = atac_df.chr_a + '_' + atac_df.start_a.map(str) + '_' + atac_df.stop_a.map(str)
    atac_count = pd.DataFrame(atac_df.groupby('TSS')['p_loc'].count())
    atac_count.columns=['num_atac_regions']
    atac_count.index.name = None

    return atac_count

In [510]:
atac_file_promoter = '../data/interim/annon/promoter_atac/promoter_GDSD3_annon.bed'
atac_file_anchor = '../data/interim/annon/anchor_atac/GDSD3_annon.bed'


In [560]:
atac_count_prom = read_atac_file(atac_file_promoter)

In [561]:
atac_count_df = read_atac_file(atac_file_anchor)
atac_count_anchor = atac_count_df.merge(anchor_annon_df_filt,how='inner', left_index=True,right_on='anchor')
atac_count_anchor = pd.DataFrame(atac_count_anchor.groupby('TSS')['num_atac_regions'].sum())
atac_count_anchor.index.name = None

In [562]:
atac_count_loop = atac_count_df.merge(loop_df_bi,how='inner',left_index=True,right_on='target')
atac_count_loop = pd.DataFrame(atac_count_loop.groupby('TSS')['num_atac_regions'].sum())
atac_count_loop.index.name = None

In [563]:
atac_count_anchor.shape, atac_count_loop.shape

((4361, 1), (8646, 1))

## E. getting loop counts

In [549]:
loop_counts_df = loop_df_bi.groupby('TSS').agg({"count":"sum","target":"count"})
loop_counts_df.columns = ['num_loop_counts','num_loops']


## F. merging count objects

In [433]:
tissue

'GDSD3'

In [553]:
print(rna_df.shape)
rna_df.index

(24811, 39)


Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A3GALT2', 'A4GALT', 'A4GNT',
       ...
       'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX',
       'ZZEF1', 'ZZZ3'],
      dtype='object', length=24811)

In [438]:
all_tfs = annon_dict.values()
len(all_tfs)

401

In [466]:
all_tf_all_tss = pd.DataFrame( index=rna_df.index)
all_tf_all_tss['exp'] = rna_df[tissue]
all_tf_all_tss.shape


(24811, 1)

In [536]:
foot_count_sum = (foot_count_pro + foot_count_anchor + foot_count_loop).fillna(0)
foot_count_comb = foot_count_anchor.add(foot_count_loop,fill_value=0)
foot_count_pro.columns = [x+'_pro' for x in foot_count_pro
                          .columns.values]
foot_count_anchor.columns = [x+'_anc' for x in foot_count_anchor.columns.values]
foot_count_loop.columns = [x+'_loop' for x in foot_count_loop.columns.values]
foot_count_comb.columns = [x+'_loop' for x in foot_count_comb.columns.values]

In [569]:
atac_count_sum = (atac_count_prom + atac_count_anchor + atac_count_loop).fillna(0)
atac_count_prom.columns = ['num_atac_regions_pro']
atac_count_comb = atac_count_anchor.add(atac_count_loop,fill_value=0)
atac_count_anchor.columns = ['num_atac_regions_anc']
atac_count_loop.columns = ['num_atac_regions_loop']
atac_count_comb.columns = ['num_atac_regions_loop']

In [554]:
# type = 'promoter': only include promoter information in the result, will include "_pro" suffixes 
all_count_pro = pd.concat([all_tf_all_tss, atac_count_prom, foot_count_pro],axis=1).fillna(0)
all_count_pro = all_count_pro[all_count_pro.index.isin(rna_df.index)]
all_count_pro.shape

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(24811, 232)

In [555]:
# type = 'anchor_loop_sep': include anchor and loop information separately, will include "_pro", "_anc" and "_loop" suffixes 
all_count_sep = pd.concat([all_tf_all_tss, loop_counts_df, atac_count_prom, atac_count_anchor, atac_count_loop, 
                           foot_count_pro, foot_count_anchor,foot_count_loop],axis=1).fillna(0)
all_count_sep = all_count_sep[all_count_sep.index.isin(rna_df.index)]
all_count_sep.shape

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


(24811, 696)

In [570]:
# type = 'anchor_loop_comb': include anchor and loop information summed, will include "_pro" and "_loop" suffixes
all_count_comb = pd.concat([all_tf_all_tss, loop_counts_df, atac_count_prom, atac_count_comb, 
                           foot_count_pro, foot_count_comb],axis=1).fillna(0)
all_count_comb = all_count_comb[all_count_comb.index.isin(rna_df.index)]
all_count_comb.shape

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


(24811, 465)

In [571]:
# type = 'sum': agg all "_pro" "_anc" and "_loop", no suffixes
all_count_sum = pd.concat([all_tf_all_tss, loop_counts_df, atac_count_sum, 
                           foot_count_sum],axis=1).fillna(0)
all_count_sum = all_count_sum[all_count_sum.index.isin(rna_df.index)]
all_count_sum.shape

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


(24811, 234)

In [523]:
# atac_count_prom
# atac_count_anchor
atac_count_loop

,num_atac_regions
A2ML1,7
A3GALT2,4
A4GALT,2
A4GNT,2
AAAS,3
...,...
ZWILCH,19
ZXDA,7
ZYX,2
hsa-mir-1199,3


In [587]:
all_count_sum_1 = all_count_sum.copy()
all_count_sum_1['tissue'] = 'test1'
all_count_sum_1.drop(['AHR','ARID5B','ARNT','ARNTL'],axis=1,inplace=True)
all_count_sum_2 = all_count_sum.copy()
all_count_sum_2['tissue'] = 'test2'
all_count_sum_2.drop(['ZNF816'],axis=1,inplace=True)
all_count_sum_1.shape, all_count_sum_2.shape

((24811, 231), (24811, 234))

In [582]:
all_count_sum_overall = pd.DataFrame()

In [589]:
pd.concat([all_count_sum_overall, all_count_sum_2]).fillna(0)

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,AHR,ARID5B,ARNT,ARNTL,ATF1,ATF2,ATF3,ATF4,ATF6,BACH1,...,ZNF680,ZNF768,ZNF770,ZNF816,ZNF85,exp,num_atac_regions,num_loop_counts,num_loops,tissue
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.045,0.0,0.0,0.0,test1
A1BG-AS1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.050,0.0,0.0,0.0,test1
A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,test1
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.925,0.0,0.0,0.0,test1
A2M-AS1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,test1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.280,0.0,0.0,0.0,test2
ZYG11B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.455,0.0,0.0,0.0,test2
ZYX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.945,0.0,17.0,1.0,test2
ZZEF1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.270,0.0,0.0,0.0,test2
